In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from ipywidgets import interact

# from conc_obj import EEGData
from eegdata_multi import EEGData
from plt import plot_psd, plot_montage
from ica import plot_ica_comp

# MNE imports
import mne
from mne.io.edf import read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

import json


***Macros***

>General use macros, importing JSON files to use as the configuration files

In [2]:
script_path = Path().resolve()
folder = (script_path / "../").resolve()

JSON_MAIN_PATH = script_path / "config_main.json"
JSON_CSP_PATH = script_path / "config_csp.json"
EVENTS_PATH = script_path / "events.json"

with open(JSON_MAIN_PATH, "r") as f:
    config_main = json.load(f)

with open(JSON_CSP_PATH, "r") as f:
    config_csp = json.load(f)

VERBOSE = config_main['verbose'].lower() == 'true'

L_FREQ = config_main['l_freq']
H_FREQ = config_main['h_freq']

N_SUBJECTS = config_main["n_subjects"]
N_COMPONENTS = config_main["n_components"]

"""
T0 corresponds to rest
T1 corresponds to onset of motion (real or imagined) of
the left fist (in runs 3, 4, 7, 8, 11, and 12)
both fists (in runs 5, 6, 9, 10, 13, and 14)
T2 corresponds to onset of motion (real or imagined) of
the right fist (in runs 3, 4, 7, 8, 11, and 12)
both feet (in runs 5, 6, 9, 10, 13, and 14)
"""


'\nT0 corresponds to rest\nT1 corresponds to onset of motion (real or imagined) of\nthe left fist (in runs 3, 4, 7, 8, 11, and 12)\nboth fists (in runs 5, 6, 9, 10, 13, and 14)\nT2 corresponds to onset of motion (real or imagined) of\nthe right fist (in runs 3, 4, 7, 8, 11, and 12)\nboth feet (in runs 5, 6, 9, 10, 13, and 14)\n'

***Initialization of EEG object***

>***(If the files are not locally stored, it will download them to the user system automatically)***

>***Use of functions like .filter_data() also is obligatory if there is no data stored locally***

In [3]:
eeg_obj = EEGData(config_main, config_csp, folder, verbose=VERBOSE)

#* Filters data and plots PSD to see differences
# eeg_obj.filter_data()
# eeg_obj.plot_psd_ba_filt(verbose=VERBOSE)

# eeg_obj.plot_psd(verbose=VERBOSE)

#* Normalizes data
# eeg_obj.normalize_data()

Loaded data:
<Raw | sample_mmi_h_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded> <Raw | sample_mmi_hf_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded>
<Raw | sample_mmi_h_filt_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded> <Raw | sample_mmi_hf_filt_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded>


**Basic information and montage plotting in 2D & 3D**
> ***The channel names can also be printed***

In [4]:

#* Plots different montages in 2D & 3D
# data = eeg_obj.get_raw_h()
# ch_names = data.info["ch_names"] 
# plot_montage(eeg_obj.montage, ch_names)

**ICA(Independent Component Analysys)**
> ***The number of components that ICA will try to sort out can be changed, it is advised to use values in the range [15-45]***

> ***Ocular artifacts are also removed, since they don't contribute to the muscular movement on this evaluation***

> ***The components can also be plotted and ocular artifacts, EOG, will be clearly visible***

In [5]:

#* Computes ICA components
# eeg_obj.decomp_ica(n_components=N_COMPONENTS, plt_show=True, verbose=VERBOSE)

#* Plot components of ICA
# plot_ica_comp(folder / config["path_ica_h"])

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

data, _ = eeg_obj.get_filt()
events, _ = eeg_obj.get_events()
event_l = config_csp["ev_mlist_eight"]
groupeve_dict = config_csp["event_dict_h"]
event_dict1 = {key: value for key, value in groupeve_dict.items() if value in event_l[0]}
print(event_dict1)

epochs = mne.Epochs(data, events, event_id=event_dict1, tmin=0.3, tmax=3.3, baseline=None, verbose=VERBOSE)
data = epochs.get_data()
# data = data.reshape(data.shape[0], -1)

labels = epochs.events[:, -1]
print(data.shape, labels.shape)

from sklearn.ensemble import VotingClassifier

svm_clf = SVC(kernel='rbf', C=100, gamma=2, probability=True)
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42)

ensemble = VotingClassifier(estimators=[('svm', svm_clf), ('rf', rf_clf)], voting='soft')

# Create a new pipeline for LDA
pipeline = Pipeline([
    ('csp', CSP(n_components=8, log=True, norm_trace=False)),
    ('ica', FastICA(n_components=32)),
    ('scaler', StandardScaler()), # StandardScaler works best
	('voting_cs', ensemble)
	# ('lda', LDA())
    # ('svm', SVC(kernel='rbf', C=100, gamma=2))
	# ('rf', RandomForestClassifier())
])

# Fit the 3D pipeline to the transformed data
pipeline.fit(data, labels)

#* Grid Search used for optimizing parameters
""" from sklearn.model_selection import GridSearchCV

param_grid = {
    'csp__n_components': [4, 6, 8],
    'svm__C': [100, 200, 300],  # Try a wider range
    'svm__gamma': [1, 2]  # Smaller gamma values might work better
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=0)
grid_search.fit(data, labels)

print("Best parameters for SVM:", grid_search.best_params_)
print("Best SVM accuracy:", grid_search.best_score_) """

from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, data, labels, cv=cv)
print("Cross-validation scores:", scores)


{'do/left_hand': 1, 'do/right_hand': 2, 'imagine/left_hand': 3, 'imagine/right_hand': 4, 'rest': 5}
Using data from preloaded Raw for 3540 events and 481 original time points ...
0 bad epochs dropped
(3540, 64, 481) (3540,)
Computing rank from data with rank=None
    Using tolerance 0.0018 (2.2e-16 eps * 64 dim * 1.3e+11  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
Estimating class=5 covariance using EMPIRICAL
Done.


/Users/Shared/42/ML/.venv/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 0.0017 (2.2e-16 eps * 64 dim * 1.2e+11  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
Estimating class=5 covariance using EMPIRICAL
Done.


/Users/Shared/42/ML/.venv/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 0.0017 (2.2e-16 eps * 64 dim * 1.2e+11  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
Estimating class=5 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0017 (2.2e-16 eps * 64 dim * 1.2e+11  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
Estimating class=5 covariance us

/Users/Shared/42/ML/.venv/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 0.0017 (2.2e-16 eps * 64 dim * 1.2e+11  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
Estimating class=5 covariance using EMPIRICAL
Done.


/Users/Shared/42/ML/.venv/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 0.0017 (2.2e-16 eps * 64 dim * 1.2e+11  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
Estimating class=5 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 0.0017 (2.2e-16 eps * 64 dim * 1.2e+11  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=1 covariance using EMPIRICAL
Done.
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=3 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
Estimating class=5 covariance us

In [7]:
#* Saves filtered and concatenated data for faster loading
# eeg_obj.save_type_data(type="raw", folder_path=folder)
# eeg_obj.save_type_data(type="filtered", folder_path=folder)
# eeg_obj.save_type_data(type="norm", folder_path=folder)
# eeg_obj.save_type_data(type="ica", folder_path=folder)